In [203]:
import sqlite3
import pandas as pd
import os
import copy
pd.options.mode.chained_assignment = None

In [204]:
process_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","02_processed","intermediate.db")
print(process_path)

/home/jupyter/Team-Prophecy/Data/02_processed/intermediate.db


In [205]:
process_connection = sqlite3.connect(process_path)

In [206]:
inputs = {    
    'prog_desc'     : "All",
    'courses'       : "All",
    'mod'           : "F2F", #F2F
    'visa'          : "F1 Visa", #F1 Visa
    'required_only' : True,
    'ExpN_eat'      : 150
}

In [207]:
# Building the model parameters
model_params = {
    'prog_desc'         : inputs['prog_desc'],
    'courses'           : inputs['courses'],
    'mod'               : inputs['mod'],
    'visa'              : inputs['visa'],
    'ExpN_eat'          : inputs['ExpN_eat'],
    'enr_hist'          : 4 if inputs['visa'] == 'F1 Visa' else 8,
    'filters'           : {
        'prog_filter'   : inputs['prog_desc'] != 'All',
        'course_filter' : inputs['courses'] != 'All',
        'visa_filter'   : inputs['visa'] != 'combined',
        'required_only' : inputs['required_only']
    },
    'groupby'           : {
        'mod_groupby'   : inputs['mod'] != 'combined'
    }
}

In [208]:
def list_to_str(l):
    """
    Create a string in the format of "('a', 'b', 'c', 'd')" from list ['a', 'b', 'c', 'd']
    """
    return str(tuple(l))

In [209]:
student_stat_query = """
    SELECT rs.reg_term_code, rs.reg_stu_id, crs, rs.sect_id, rs.reg_new_ret_stu, rs.reg_final_status
    FROM registration_status rs
    WHERE 1=1 
"""
student_details_query = """
    SELECT stu_admit_term_code, stu_college, stu_deg_level, stu_dept, stu_id, stu_res, stu_prog, stu_visa, stu_bam
    FROM student_details 
    WHERE 1=1 
"""

program_course_offerings = """
    SELECT * FROM PROGRAM_COURSE_OFFERINGS 
"""

# Program Filter
if model_params['filters']['prog_filter']:
    student_details_query += f"AND stu_prog = '{model_params['prog_desc']}' "

# Course Filter
if model_params['filters']['course_filter']:
    student_stat_query += f"AND crs IN {list_to_str(model_params['courses'])} "

# Visa Filter
if model_params['filters']['visa_filter']:
    student_details_query += f"AND stu_visa = '{model_params['visa']}' "

In [210]:
student_stat = pd.DataFrame(process_connection.execute(student_stat_query).fetchall(),columns=["reg_term_code", "student_id", "crs", "sect_id", "returning_student", "reg_status"])

#WHERE reg_final_status IN ('W','R')
#GROUP BY reg_term_code, crs, sect_id, reg_final_status

student_details = pd.DataFrame(process_connection.execute(student_details_query).fetchall(),columns=["reg_term_code","stu_college","stu_deg_level","stu_dept","student_id",
                                                                                                     "stu_res","stu_prog","stu_visa","stu_bam"])

pco_df = pd.DataFrame(process_connection.execute(program_course_offerings),columns=["stu_prog","crs","required"])

total_stat = student_stat.merge(student_details, on=["reg_term_code","student_id"], how="inner").fillna(0)
total_stat = total_stat.merge(pco_df, on=["stu_prog","crs"])

In [211]:
required_dictionary = pco_df.drop_duplicates().set_index(["stu_prog","crs"])["required"].to_dict()

In [222]:
#required_dictionary

In [213]:
if model_params['filters']['required_only']:
    total_stat = total_stat.loc[total_stat["required"] == 1,:]

In [214]:
total_student_population = total_stat[["reg_term_code"]].groupby("reg_term_code").count().reset_index()

In [215]:
reg_term_values = dict(total_stat.groupby(["reg_term_code"])["reg_term_code"].count())

In [216]:
#program_limit_values = dict(total_stat.groupby(["stu_prog"])["stu_prog"].count())

In [218]:
req_courses_context = total_stat.loc[total_stat["reg_status"] == "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
#nonregistered_courses = total_courses.loc[total_courses["reg_status"] != "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

Keep in mind: Our registered values will be dependent on what's in our subset.

In [19]:
#req_courses_context = registered_courses.merge(ref_course[["reg_term_code", "crs","sect_id", "cum_total_enrollment"]], on=["reg_term_code", "crs","sect_id"], how="inner").drop_duplicates()
#req_courses_context.loc[req_courses_context["cum_total_enrollment"] < req_courses_context["reg_status"],["cum_total_enrollment"]]  = req_courses_context["reg_status"]
#req_courses_context.columns

In [219]:
reg_term_courses_context = req_courses_context.groupby(["reg_term_code"])

In [220]:
from ortools.linear_solver import pywraplp
from itertools import chain

What we're going to do
-> One thing will be a script file that runs automatically behind the scenes
-> In the frontend, what we're going to do is define an input dictionary
    This is the code that we want: Only international students, and we'll have 

In [41]:
all_semesters = list(reg_term_values.keys())
#program_limit_values <- included because we need a way to constraint students
all_pcs_list = list(req_courses_context[["stu_prog","crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
all_pcs_dict = {}
#References what program contains which class
pc_dict = {}
cs_dict = {}
for t_pcs in all_pcs_list:
    if t_pcs[0] not in all_pcs_dict:
        pc_dict[t_pcs[0]] = []
        all_pcs_dict[t_pcs[0]] = {}
    pc_dict[t_pcs[0]].append(t_pcs[1])
    if t_pcs[1] not in all_pcs_dict[t_pcs[0]]:
        cs_dict[t_pcs[1]] = []
        all_pcs_dict[t_pcs[0]][t_pcs[1]] = []
    cs_dict[t_pcs[1]].append(t_pcs[2])
    all_pcs_dict[t_pcs[0]][t_pcs[1]].append(t_pcs[2])

In [137]:
#req_courses_context
#req_courses_context[["reg_term_code","stu_prog","crs","sect_id","reg_status"]]

In [43]:
req_courses_context["stu_prog"].drop_duplicates().tolist()

['EC-MS-AIT', 'EC-MS-CEIE', 'EC-MS-DAEN', 'EC-MS-ISYS']

In [25]:
#Constraints
#total_student_population
#program_limit_values

Fall 2020
150 -> 80 Program_1, 40 Program_2, 30 Program_3
50
1 student -> 15 credits = 3-4 classes

In [124]:
req_mean = req_courses_context.groupby(["crs","sect_id"]).mean("reg_status").reset_index()
req_med = req_courses_context.groupby(["crs","sect_id"]).median("reg_status").reset_index()
req_mean = req_mean.rename(columns={"reg_status": "mean"})
req_med = req_med.rename(columns={"reg_status": "median"})
req_avg = req_mean.merge(req_med,on=["crs","sect_id"],how="inner")
req_avg["avg"] = round(req_avg[["mean","median"]].max(axis=1))
req_avg = req_avg[["crs","sect_id","avg"]]

In [125]:
course_sect_avg_dict = req_avg.set_index(["crs","sect_id"])["avg"].to_dict()

In [126]:
req_mean = req_courses_context.groupby(["crs"]).mean("reg_status").reset_index()
req_med = req_courses_context.groupby(["crs"]).median("reg_status").reset_index()
req_mean = req_mean.rename(columns={"reg_status": "mean"})
req_med = req_med.rename(columns={"reg_status": "median"})
req_avg = req_mean.merge(req_med,on=["crs"],how="inner")
req_avg["avg"] = round(req_avg[["mean","median"]].max(axis=1))
req_avg = req_avg[["crs","avg"]]

In [127]:
course_avg_dict = req_avg.set_index(["crs"])["avg"].to_dict()

In [136]:
def modelSolve(a_sem,cxt_grp,storage_obj,prior_student_info):
    #WE HAVE RATIOS HERE IN CASE WE NEED TO DO MORE ANALYSIS OR MATCH IT TO 1
    #
    #These will be our constraints
    CAP_s = model_params['ExpN_eat']
    
    cxt_grp["reg_status"] = cxt_grp["reg_status"]/reg_term_values[a_sem]
    #total_students_list = cxt_grp["cum_total_enrollment"]/reg_term_values[a_sem]
    pcs_weighting_ratio = cxt_grp[["reg_term_code","stu_prog","crs","sect_id","reg_status"]].set_index(["crs"])["reg_status"].to_dict()
    pcs_weighting_ratio = {k : v for k,v in pcs_weighting_ratio.items()} #"("+",".join(k)+")"
    #Now that we have the ratios, we can begin with the objective function.
    #
    program_ref = {}
    course_ref = {}
    
    program_w_ref = {}
    course_w_ref = {}
    course_w_ref = {}
    
    course_student_ref = {}
    course_student_w_ref = {}
    
    #Total Decision and Constant Variables Used
    decision_vars = {}
    const_vars = {}
    
    #Student ID Decision Variables
    x_decision_vars = {}
    w_decision_vars = {}
    a_decision_vars = {}
    b_decision_vars = {}
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Cannot get solver")
        return
        
    cs_l = cxt_grp["crs"].drop_duplicates().tolist()

    #
    s_inf = solver.infinity()
    
    all_cs = list(cxt_grp[["crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
    
    for cs in all_cs:
        if cs[0] not in course_ref.keys():
            course_ref[cs[0]] = []
            course_w_ref[cs[0]] = []
            
        if cs[0] not in course_student_ref.keys():
            course_student_ref[cs[0]] = {}
            course_student_w_ref[cs[0]] = {}
        
        for student_id in range(1,CAP_s):
            #We'll have two primary decision variables: Waitlist and Student
            #This will be a minimization function for our purposes, as we want all students
            # to be included.
            
            x_name = f"x({student_id},{a_sem})"
            w_name = f"w({student_id},{a_sem})"
            
            #Declares the decision variable
            
            
            decision_vars[x_name] = solver.IntVar(0,1,x_name)
            decision_vars[w_name] = solver.IntVar(0,1,w_name)
            
            if student_id not in x_decision_vars:
                x_decision_vars[student_id] = []
            if student_id not in w_decision_vars:
                w_decision_vars[student_id] = []
            x_decision_vars[student_id].append(decision_vars[x_name])
            w_decision_vars[student_id].append(decision_vars[w_name])
            
            program_ref[cs[0]].append(decision_vars[x_name])
            program_w_ref[cs[0]].append(decision_vars[w_name])
            
            
            course_ref[cs[1]].append(decision_vars[x_name])
            course_w_ref[cs[1]].append(decision_vars[w_name])
            
            course_student_ref[cs[1]][student_id] = decision_vars[x_name]
            course_student_w_ref[cs[1]][student_id] = decision_vars[w_name]
    
        const_vars[f"01_course_with_w_{cs}_constr_ratio"] = solver.Add(pcs_weighting_ratio[cs]*solver.Sum(course_ref[cs]) <= course_avg_dict[cs] + solver.Sum(course_w_ref[cs]), name=f"01_course_with_w_{cs}_constr_ratio")
        const_vars[f"02_course_{cs}_constr"] = solver.Add(solver.Sum(course_ref[cs]) <= course_avg_dict[cs], name=f"02_course_with_w_{cs}_constr")
        for student_id in range(1,reg_term_values[a_sem]): 
            const_vars[f"03_course_{cs}_sections_waitlist_constr"] = solver.Add(course_student_w_ref[cs][student_id] <= 1, name=f"03_course_{cs}_sections_waitlist_constr")
            const_vars[f"04_course_{cs}_sections_constr"] = solver.Add(course_student_ref[cs][student_id] <= 1, name=f"04_course_{cs}_sections_constr")
            
            const_vars[f"05_one_regstat_{student_id}"] = solver.Add(course_student_ref[cs][student_id] + course_student_w_ref[cs][student_id] == 1, name=f"05_one_regstat_{student_id}")
            
            a_name = f"a({cs},{a_sem})"
            b_name = f"b({cs},{a_sem})"
            a_decision_vars[a_name] = solver.IntVar(0,s_inf,a_name)
            b_decision_vars[b_name] = solver.IntVar(0,s_inf,b_name)
            
            const_vars[f"06_student_bounds_{student_id}"] = solver.Add(solver.Sum(x_decision_vars[student_id]) <= 3, name=f"06_student_bounds_{student_id}")
            #const_vars[f"03_student_w_bounds_{student_id}"] = solver.Add(solver.Sum(w_decision_vars[student_id]) <= 3, name=f"03_student_bounds_w_{student_id}")
        const_vars[f"07_organization_course_{cs}_const"] = solver.Add(solver.Sum([course_student_ref[cs][student_id] for student_id in range(1,reg_term_values[a_sem])]) == a_decision_vars[a_name],name=f"07_organization_course_{cs}_const")
        const_vars[f"08_organization_wait_course_{cs}_const"] = solver.Add(solver.Sum([course_student_w_ref[cs][student_id] for student_id in range(1,reg_term_values[a_sem])]) == b_decision_vars[b_name],name=f"08_organization_wait_course_{cs}_const")
        
    
    # Minimization function 
    # -> For later * a_decision_vars[student_id]
    
    solver.Minimize(solver.Sum([solver.Sum(w_decision_vars[student_id])
                                for student_id in range(1,reg_term_values[a_sem])]))
    solver.Solve()

In [69]:
prior_student_info = {}
for a_sem in all_semesters:
    cxt_grp = reg_term_courses_context.get_group(a_sem)
    prg_list = cxt_grp["stu_prog"].drop_duplicates().tolist()
    for p in prg_list:
        modelSolve(a_sem,cxt_grp.loc[cxt_grp["stu_prog"] == p,:],prior_student_info)
        break
    break

KeyboardInterrupt: 

In [27]:
solver.NumVariables()

2744

In [28]:
solver.NumConstraints()

2800

In [29]:
print('Objective value =', solver.Objective().Value())

Objective value = 636.0


In [496]:
#for i in list(x_decision_vars.values()):
#    print(f"{i}")

In [30]:
for i in list(a_decision_vars.values()):
    print(f"{i.name()} -> {i.SolutionValue()}")

a(AIT524,202010) -> 2.0
a(AIT542,202010) -> 2.0
a(AIT602,202010) -> 1.0
a(AIT673,202010) -> 1.0
a(AIT677,202010) -> 3.0
a(AIT716,202010) -> 1.0
a(CEIE524,202010) -> 1.0
a(CEIE575,202010) -> 3.0
a(CEIE576,202010) -> 14.0
a(CEIE578,202010) -> 3.0
a(CEIE605,202010) -> 13.0
a(CEIE679,202010) -> 1.0
a(CEIE795,202010) -> 4.0
a(DAEN500,202010) -> 1.0


In [31]:
for i in list(b_decision_vars.values()):
    print(f"{i.name()} -> {i.SolutionValue()}")

b(AIT524,202010) -> 47.0
b(AIT542,202010) -> 47.0
b(AIT602,202010) -> 48.0
b(AIT673,202010) -> 48.0
b(AIT677,202010) -> 46.0
b(AIT716,202010) -> 48.0
b(CEIE524,202010) -> 48.0
b(CEIE575,202010) -> 46.0
b(CEIE576,202010) -> 35.0
b(CEIE578,202010) -> 46.0
b(CEIE605,202010) -> 36.0
b(CEIE679,202010) -> 48.0
b(CEIE795,202010) -> 45.0
b(DAEN500,202010) -> 48.0


In [ ]:
process_connection.execute("CREATE TABLE results_table("
                       "rec_id INTEGER PRIMARY KEY AUTOINCREMENT DEFAULT 0, "
                       "textual_context TEXT NOT NULL, "
                       "number_of_students INTEGER NOT NULL"
                       ");")
process_connection.commit()

**NOW WE CAN GET INTO THE INTERESTING STUFF**

This is for when I have more data; only focusing on what I know for certain given the total number of students available.

In [ ]:
waitlist_courses = total_courses.loc[total_courses["reg_status"] == "W",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
dropped_courses = total_courses.loc[total_courses["reg_status"] == "D",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

In [ ]:
waitlist_courses = waitlist_courses.rename(columns={'reg_status':'waitlist'})
required_courses = required_courses.rename(columns={'reg_status':'registered'})
dropped_courses = dropped_courses.rename(columns={'reg_status':'dropped'})

In [ ]:
total_courses = required_courses.merge(waitlist_courses, on=["reg_term_code", "stu_prog","crs","sect_id"], how="left").fillna(0)
total_courses = total_courses.merge(dropped_courses, on=["reg_term_code", "stu_prog","crs","sect_id"], how="left").fillna(0)